In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sys
import os

In [3]:
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
path_to_datasets = os.path.join(project_root, 'data', 'external', 'archive')
orders = pd.read_csv(f"{path_to_datasets}/olist_order_items_dataset.csv")
purchases = pd.read_csv(f"{path_to_datasets}/olist_orders_dataset.csv", parse_dates=['order_purchase_timestamp'])
products = pd.read_csv(f"{path_to_datasets}/olist_products_dataset.csv")
orders.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [8]:
dataset = orders[['order_id', 'product_id', 'price']].copy()\
    .merge(purchases[['order_id', 'order_purchase_timestamp']].copy(), how='left', on='order_id')\
    .merge(products[['product_id', 'product_category_name']].copy(), how='left', on='product_id')
dataset.head()

,order_id,product_id,price,order_purchase_timestamp,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.90,2017-09-13 08:59:02,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.90,2017-04-26 10:53:06,pet_shop
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.00,2018-01-14 14:33:31,moveis_decoracao
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.99,2018-08-08 10:00:35,perfumaria
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.90,2017-02-04 13:57:51,ferramentas_jardim


Selecting category with the most purchases

In [9]:
category_counts = dataset["product_category_name"].value_counts()

# Find the category with the most purchases
most_purchased_category = category_counts.idxmax()
category_counts

product_category_name
cama_mesa_banho                  11115
beleza_saude                      9670
esporte_lazer                     8641
moveis_decoracao                  8334
informatica_acessorios            7827
                                 ...  
cds_dvds_musicais                   14
la_cuisine                          14
pc_gamer                             9
fashion_roupa_infanto_juvenil        8
seguros_e_servicos                   2
Name: count, Length: 73, dtype: int64

In [10]:
dataset = dataset[dataset["product_category_name"] == most_purchased_category]
dataset.head()

,order_id,product_id,price,order_purchase_timestamp,product_category_name
12,0006ec9db01a64e59a68b2c340bf65a7,99a4788cb24856965c36a24e339b6058,74.0,2018-07-24 17:04:17,cama_mesa_banho
20,000e63d38ae8c00bbcb5a30573b99628,553e0e7590d3116a072507a3635d2877,47.9,2018-03-23 19:48:26,cama_mesa_banho
37,00169e31ef4b29deaae414f9a5e95929,b10eba910a974df70b8a12d0665cdb9e,39.0,2018-01-16 09:26:39,cama_mesa_banho
47,001c85b5f68d2be0cb0797afc9e8ce9a,84f456958365164420cfc80fbe4c7fab,99.0,2017-11-24 19:19:18,cama_mesa_banho
51,001dbc16dc51075e987543d23a0507c7,777d2e438a1b645f3aec9bd57e92672c,69.9,2017-01-28 13:17:57,cama_mesa_banho


Selecting top5 products

In [11]:
# Count the occurrences of each product_id
product_counts = dataset["product_id"].value_counts()
product_counts

product_id
99a4788cb24856965c36a24e339b6058    488
f1c7f353075ce59d8a6f3cf58f419c9c    154
06edb72f1e0c64b14c5b79353f7abea3    143
ec2d43cc59763ec91694573b31f1c29a    131
84f456958365164420cfc80fbe4c7fab    111
                                   ... 
f1f8a9e252e1f78ae6022bb2d40c392f      1
4cee4c57bf2e00c0c0e10918245491c1      1
8afa66574cf6cd61a7bff6af5494b37a      1
1aa5ffb5e73a8635a5d4619a96d012fe      1
1303e8674814cde0e8cfe7d796d6a544      1
Name: count, Length: 3029, dtype: int64

In [13]:
# Get the top 5 product_ids with the most purchases
top_5_products = product_counts.head(5).index

    # Filter the dataset to include only these top 5 products
top_5_dataset = dataset[dataset["product_id"].isin(top_5_products)]

top_5_dataset = top_5_dataset.reset_index(drop=True)
top_5_dataset.drop("product_category_name", axis=1, inplace=True)
top_5_dataset

,order_id,product_id,price,order_purchase_timestamp
0,0006ec9db01a64e59a68b2c340bf65a7,99a4788cb24856965c36a24e339b6058,74.00,2018-07-24 17:04:17
1,001c85b5f68d2be0cb0797afc9e8ce9a,84f456958365164420cfc80fbe4c7fab,99.00,2017-11-24 19:19:18
2,005059edee63c8c708ba61910793b31b,84f456958365164420cfc80fbe4c7fab,92.00,2018-02-05 18:25:51
3,005059edee63c8c708ba61910793b31b,84f456958365164420cfc80fbe4c7fab,92.00,2018-02-05 18:25:51
4,009905140e9f8cc35d5be897937381db,06edb72f1e0c64b14c5b79353f7abea3,39.99,2018-03-15 12:33:45
...,...,...,...,...
1022,fec4d19f04178e4adbe16ebfc8c7ebd7,f1c7f353075ce59d8a6f3cf58f419c9c,205.00,2017-07-11 13:33:59
1023,fee19a0dc7358b6962a611cecf6a37b4,f1c7f353075ce59d8a6f3cf58f419c9c,195.00,2017-08-29 21:51:54
1024,ff00a56fe9475a175cd651d77c707a09,99a4788cb24856965c36a24e339b6058,89.90,2017-11-24 00:08:10
1025,ff79282c8f4c58df02fde009ad02d009,99a4788cb24856965c36a24e339b6058,86.00,2018-04-19 12:29:54
